In this first cell I imported and installed all necessary libraries for the project development

In [5]:
import requests # library to handle requests

import pandas as pd

!pip3 install beautifulsoup4 requests
import requests
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


After, I got web page information and analysed data structure

In [6]:
# set information data page and parse data
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html5lib')

# Analyse html
#(soup.prettify())

table=soup.find(class_="wikitable sortable")

Finally I iterated trought the data frame to remove missing information from the dataset

In [7]:
table_contents=[]
table=soup.find('table')

for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)  

And then I converted the table obtained into a dataframe

In [8]:
# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

#Visualize dataframe

df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [9]:
#Use method shape to determene the number of rows of the dataframe

print(len(df))

103


PART 2 / WK 3 

importing the necessary modules

In [10]:
!pip install pgeocode
import pgeocode

Convet the column Postalcode from previous created dataframe into a list. Initialize longitude and latitude list and run trought then assgining the values of latitude obtained by geolocator.

In [11]:
pgeocode.Nominatim('ca')
geolocator = pgeocode.Nominatim('ca')
postal_codes = df['PostalCode'].tolist()
latitudes = []
longitudes = []
for i, postal_code in enumerate(postal_codes):
    # initialize your variable to None
    #print(f'--Getting Postal Code: {postal_code}')
    g = geolocator.query_postal_code(postal_code)
    
    if not g.empty:
        #print(f'Postal Code {postal_code} has been retrieved. {len(postal_codes) - (i + 1)} codes left')
        latitudes.append(g.latitude)
        longitudes.append(g.longitude)

Append to data frame the lists with latitude and longitude values

In [12]:
df['Latitude'] = latitudes
df['Longitude'] = longitudes

In [13]:
df= df.dropna(how='all')  #Remove all Nan values from dataframe
df 

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Queen's Park,Ontario Provincial Government,43.6641,-79.3889
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6518,-79.5076
99,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
100,M7Y,East Toronto Business,Enclave of M4L,43.7804,-79.2505
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6325,-79.4939


PART 3 - BEGINNING

In [17]:
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [18]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [24]:
df=df.dropna(subset=['Longitude'])
df=df.dropna(subset=['Latitude'])

In [25]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

In [26]:
CLIENT_ID = 'VILYW5KME0TMMF1EBNTJ0QUL0ZPVNHBVLQP4HZUHQITNEKHF' # your Foursquare ID
CLIENT_SECRET = 'OXPTUCR51DWXTENH0JTWLDSRP533Y4CKMQWTW23VFX5GI4ZE' # your Foursquare Secret
VERSION = '20210725' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VILYW5KME0TMMF1EBNTJ0QUL0ZPVNHBVLQP4HZUHQITNEKHF
CLIENT_SECRET:OXPTUCR51DWXTENH0JTWLDSRP533Y4CKMQWTW23VFX5GI4ZE


In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [48]:
for index, row in df.iteritems():
    toronto_venues = getNearbyVenues(df['Neighborhood'], df['Latitude'], df['Longitude'] )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

Runnymede, Swansea
Clarks Corners, Tam O'Shanter, Sullivan
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
New Toronto, Mimico South, Humber Bay Shores
South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens
Steeles West, L'Amoreaux West
Rosedale
Enclave of M5E
Alderwood, Long Branch
Clairville, Humberwood, Woodbine Downs, West Humber, Kipling Heights, Rexdale, Elms, Tandridge, Old Rexdale
Upper Rouge
St. James Town, Cabbagetown
First Canadian Place, Underground city
The Kingsway, Montgomery Road, Old Mill North
Church and Wellesley
Enclave of M4L
Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East
Mimic

KeyError: 'groups'

In [49]:
print(toronto_venues.shape)
toronto_venues.head()

(2158, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7545,-79.3300,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.7545,-79.3300,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.7545,-79.3300,TTC stop #8380,43.752672,-79.326351,Bus Stop
3,Parkwoods,43.7545,-79.3300,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,Victoria Village,43.7276,-79.3148,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [50]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",6,6,6,6,6,6
"Bathurst Manor, Wilson Heights, Downsview North",7,7,7,7,7,7
Bayview Village,5,5,5,5,5,5
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
...,...,...,...,...,...,...
"Willowdale, Newtonbrook",1,1,1,1,1,1
Woburn,2,2,2,2,2,2
Woodbine Heights,6,6,6,6,6,6


In [51]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 255 uniques categories.


In [52]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
toronto_onehot.shape

(2158, 255)

In [54]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
95,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
96,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0
97,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
toronto_grouped.shape

(99, 255)

In [56]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1  Latin American Restaurant  0.25
2               Skating Rink  0.25
3            Badminton Court  0.25
4                  Pet Store  0.00


----Alderwood, Long Branch----
               venue  freq
0  Convenience Store  0.17
1                Pub  0.17
2     Sandwich Place  0.17
3        Coffee Shop  0.17
4                Gym  0.17


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Coffee Shop  0.14
1                Pizza Place  0.14
2   Mediterranean Restaurant  0.14
3        Fried Chicken Joint  0.14
4  Middle Eastern Restaurant  0.14


----Bayview Village----
         venue  freq
0  Flower Shop   0.2
1         Park   0.2
2      Dog Run   0.2
3        Trail   0.2
4  Gas Station   0.2


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.08
1         Coffee Shop  0.08
2         Pizza Place  0.08
3

                                      venue  freq
0  Residential Building (Apartment / Condo)   0.5
1                                      Park   0.5
2                               Yoga Studio   0.0
3                         Mobile Phone Shop   0.0
4                             Movie Theater   0.0


----Humber Summit----
                        venue  freq
0  Construction & Landscaping  0.33
1                 Music Store  0.33
2         Rental Car Location  0.33
3                 Men's Store  0.00
4               Metro Station  0.00


----Humberlea, Emery----
            venue  freq
0    Home Service  0.14
1  Discount Store  0.14
2       Nightclub  0.14
3   Grocery Store  0.14
4     Coffee Shop  0.14


----Humewood-Cedarvale----
          venue  freq
0         Field  0.11
1  Tennis Court  0.11
2  Hockey Arena  0.11
3          Park  0.11
4    Playground  0.11


----India Bazaar, The Beaches West----
                  venue  freq
0             Pet Store  0.06
1                   Gym  0.

                venue  freq
0    Greek Restaurant  0.20
1      Ice Cream Shop  0.06
2  Italian Restaurant  0.06
3                Café  0.06
4         Yoga Studio  0.03


----The Kingsway, Montgomery Road, Old Mill North----
              venue  freq
0              Bank  0.12
1    Breakfast Spot  0.12
2            Bakery  0.12
3  Sushi Restaurant  0.12
4               Bar  0.06


----Thorncliffe Park----
                venue  freq
0   Indian Restaurant  0.16
1   Afghan Restaurant  0.08
2  Turkish Restaurant  0.08
3         Coffee Shop  0.08
4      Sandwich Place  0.08


----Toronto Dominion Centre, Design Exchange----
                venue  freq
0         Coffee Shop  0.10
1               Hotel  0.08
2                Café  0.05
3  Seafood Restaurant  0.03
4         Salad Place  0.03


----University of Toronto, Harbord----
                   venue  freq
0                   Café  0.14
1                 Bakery  0.07
2    Japanese Restaurant  0.07
3              Bookstore  0.07
4  College

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [59]:
import numpy as np

In [60]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Latin American Restaurant,Skating Rink,Badminton Court,Pet Store,Movie Theater,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
1,"Alderwood, Long Branch",Convenience Store,Pub,Sandwich Place,Coffee Shop,Gym,Pizza Place,Other Great Outdoors,Moving Target,Metro Station,Mexican Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Pizza Place,Mediterranean Restaurant,Fried Chicken Joint,Middle Eastern Restaurant,Deli / Bodega,Grocery Store,Yoga Studio,Movie Theater,Moroccan Restaurant
3,Bayview Village,Flower Shop,Park,Dog Run,Trail,Gas Station,Yoga Studio,Modern European Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Pizza Place,Italian Restaurant,Sushi Restaurant,Restaurant,Thai Restaurant,Pub,Greek Restaurant,Grocery Store


In [66]:
!pip3 install KMeans
from sklearn.cluster import KMeans

In [71]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [84]:
neighborhoods_venues_sorted.drop(['Cluster Labels'], axis=1, inplace=True)

In [85]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how ='inner')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7545,-79.3300,0,Food & Drink Shop,Park,Fast Food Restaurant,Bus Stop,Moving Target,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
1,M4A,North York,Victoria Village,43.7276,-79.3148,0,French Restaurant,Hockey Arena,Pizza Place,Park,Intersection,Portuguese Restaurant,Coffee Shop,Modern European Restaurant,Movie Theater,Moroccan Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,0,Coffee Shop,Breakfast Spot,Yoga Studio,Gym / Fitness Center,Spa,Sushi Restaurant,Beer Store,Restaurant,Event Space,Thai Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,0,Clothing Store,Coffee Shop,Women's Store,Restaurant,Shoe Store,Bakery,Sandwich Place,Toy / Game Store,Food Court,Cosmetics Shop
4,M7A,Queen's Park,Ontario Provincial Government,43.6641,-79.3889,0,Gym,Sushi Restaurant,Theater,Beer Bar,Martial Arts School,Bubble Tea Shop,Burrito Place,Mexican Restaurant,Coffee Shop,Creperie


In [78]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [86]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [87]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Food & Drink Shop,Park,Fast Food Restaurant,Bus Stop,Moving Target,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
1,North York,0,French Restaurant,Hockey Arena,Pizza Place,Park,Intersection,Portuguese Restaurant,Coffee Shop,Modern European Restaurant,Movie Theater,Moroccan Restaurant
2,Downtown Toronto,0,Coffee Shop,Breakfast Spot,Yoga Studio,Gym / Fitness Center,Spa,Sushi Restaurant,Beer Store,Restaurant,Event Space,Thai Restaurant
3,North York,0,Clothing Store,Coffee Shop,Women's Store,Restaurant,Shoe Store,Bakery,Sandwich Place,Toy / Game Store,Food Court,Cosmetics Shop
4,Queen's Park,0,Gym,Sushi Restaurant,Theater,Beer Bar,Martial Arts School,Bubble Tea Shop,Burrito Place,Mexican Restaurant,Coffee Shop,Creperie
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,0,Hotel,Coffee Shop,Café,Restaurant,Japanese Restaurant,Gym,Seafood Restaurant,Asian Restaurant,Steakhouse,Salad Place
98,Etobicoke,0,Bank,Breakfast Spot,Bakery,Sushi Restaurant,Bar,Liquor Store,Restaurant,Pub,Boutique,Smoke Shop
99,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Yoga Studio,Hotel,Men's Store,Mediterranean Restaurant,Bar
100,East Toronto Business,0,Restaurant,Coffee Shop,Yoga Studio,Japanese Restaurant,Breakfast Spot,Martial Arts School,Bookstore,Sushi Restaurant,Bank,Italian Restaurant


In [88]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Scarborough,1,Convenience Store,Korean BBQ Restaurant,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant
27,North York,1,Residential Building (Apartment / Condo),Park,Yoga Studio,Mobile Phone Shop,Movie Theater,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop
35,East York/East Toronto,1,Park,Convenience Store,Intersection,Modern European Restaurant,Moving Target,Movie Theater,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Miscellaneous Shop
61,Central Toronto,1,Photography Studio,Park,Yoga Studio,Moving Target,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
66,North York,1,Convenience Store,Park,Museum,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop
69,West Toronto,1,Residential Building (Apartment / Condo),Park,Yoga Studio,Mobile Phone Shop,Movie Theater,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop
71,Scarborough,1,Convenience Store,Auto Garage,Music Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant
101,Etobicoke,1,Convenience Store,Park,Baseball Field,Museum,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop


In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,2,Home Service,Museum,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant
62,Central Toronto,2,Home Service,Museum,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant


In [90]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,North York,3,Basketball Court,Bakery,Yoga Studio,Museum,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop
65,Scarborough,3,Bakery,Asian Restaurant,Molecular Gastronomy Restaurant,Museum,Moving Target,Movie Theater,Moroccan Restaurant,Monument / Landmark,Yoga Studio,Music Store


In [91]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,4,Baseball Field,Yoga Studio,Medical Center,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant
